# Self Study 3

This self study concludes our first "miniproject" on crawling and search. The tasks for this self study are:
- modify/extend the inverted index you constructed in the previous self study to contain for all postings the term frequencies (if your documents are just the titles of the web pages, you will see very few term frequencies larger than 1, but do not worry about that).
- calculate the idf values for all terms, and also include them in your index (cf. slide 3.20 for a schematic view)
- implement ranked retrieval as described on slides 3.19 and 3.20 for the ntc.bnc similarity metric 

In [4]:
import requests
from bs4 import BeautifulSoup
from urllib.robotparser import RobotFileParser
from nltk.stem.snowball import SnowballStemmer
import string
import nltk

titles = []
corpus = []
postings = []

numOfArticles = 10

dstemmer=SnowballStemmer("danish")

#construct the vocabulary of terms for your corpus (corpus)
#build an 'inverted' index for your vocabulary (postings)

def getTitles(link):
    titles = []

    rp=RobotFileParser()
    rp.set_url(link)
    rp.read()
    r=requests.get(link)

    r_parse = BeautifulSoup(r.text, 'html.parser')
    r_parse.find('title').string
    for i, a in enumerate(r_parse.find_all('a')):
        if(i == numOfArticles): break
        _link = a['href']
        if(_link == '#main'):   continue
        if(_link[0] == '/'):    _link = link+_link[1:]
        titles.append(_getTitles(_link))
    return titles

def _getTitles(link):
    r=requests.get(link)
    r_parse = BeautifulSoup(r.text, 'html.parser')
    return r_parse.find('title').string

def remove_non_ascii(a_str):
    ascii_chars = set(string.printable)

    return ''.join(
        filter(lambda x: x in ascii_chars, a_str)
    )

def tokenizeAndStemTitles(titles):
    _invertedIndex = []
    tokens = []
    for i, title in enumerate(titles):
        _tokens=nltk.word_tokenize(title)
        ps=nltk.PorterStemmer()
        for t in _tokens:
            s = ps.stem(t)
            s = remove_non_ascii(s)
            if(s == ''): continue
            if s not in tokens: tokens.append(s)

            flag = 0
            for el in _invertedIndex:
                if el['vocabulary'] == s:
                    el['postings'].append(i)
                    flag = 1

            if flag == 0: _invertedIndex.append(dict(vocabulary=s, postings=[i]))

    return tokens, _invertedIndex

titles = getTitles('https://www.aau.dk/')

corpus, invertedIndex = tokenizeAndStemTitles(titles)
print("corpus:", corpus)
print("inverted matrix:", invertedIndex)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1002)>